# Trade Feeds Demoes (FX Spot Orders and Trades)
#### I use real feeds from MOEX, they are over 250M+ and come packed, we have to unpack them first

In [1]:
%%sh
gunzip /opt/app/data/data/Trades/*.gz

#### You should see these two files in the data/Trades folder now

In [2]:
%%sh
ls -la -h /opt/app/data/data/Trades/

total 260644
drwxr-xr-x    1 root     root        4.0K Apr 13 16:42 .
drwxr-xr-x    1 root     root        4.0K Apr 13 01:33 ..
-rw-r--r--    1 root     root      253.2M Apr 13 16:42 TopofBook20130206.txt
-rw-r--r--    1 root     root        1.3M Apr 13 16:42 TradeLog20130206.txt


#### Let's import the feed into a dataframe
It's 6M+ records and will take some time

In [1]:
import pandas as pd
import numpy as np

_orders = '/opt/app/data/data/Trades/TopofBook20130206.txt'
_trades = '/opt/app/data/data/Trades/TradeLog20130206.txt'
#%time orders = pd.read_csv(_orders, low_memory=False)
%time trades = pd.read_csv(_trades, low_memory=False)

CPU times: user 30 ms, sys: 0 ns, total: 30 ms
Wall time: 44 ms


#### Let's inspect the dataframe

In [3]:
trades.shape

TRADENO          int64
SECCODE         object
TIME             int64
BUYORDERNO       int64
SELLORDERNO      int64
PRICE          float64
VOLUME           int64
dtype: object

In [4]:
trades.dtypes

TRADENO          int64
SECCODE         object
TIME             int64
BUYORDERNO       int64
SELLORDERNO      int64
PRICE          float64
VOLUME           int64
dtype: object

#### As you can see above, some of the column types are object and we need to convert them to ones that will allow better manipulation

In [6]:
%time trades.TIME = pd.to_datetime(trades.TIME, format='%H%M%S%f')
#%time orders.TIME = pd.to_datetime(orders.TIME, format='%H%M%S%f')

CPU times: user 120 ms, sys: 10 ms, total: 130 ms
Wall time: 128 ms


#### We converted the datetime column to proper format, let's see what we got:

In [8]:
print(trades.dtypes)

TRADENO                 int64
SECCODE                object
TIME           datetime64[ns]
BUYORDERNO              int64
SELLORDERNO             int64
PRICE                 float64
VOLUME                  int64
dtype: object


In [9]:
trades.head(5)

,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME
0,76139178,USD000UTSTOM,1900-01-01 10:00:00.000,79,4,30.0490,1000
1,76139179,USD000000TOD,1900-01-01 10:00:02.900,13453,3846,29.9990,200000
2,76139180,USD000000TOD,1900-01-01 10:00:02.900,13453,13448,29.9990,100000
3,76139181,USD000000TOD,1900-01-01 10:00:02.900,13453,13486,29.9995,150000
4,76139182,USD000000TOD,1900-01-01 10:00:03.200,15269,15271,29.9900,126000


#### Let's see instruments that were traded in this session

In [10]:
%time trades.SECCODE.unique()

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 6.98 ms


array(['USD000UTSTOM', 'USD000000TOD', 'EUR_RUB__TOM', 'EUR_RUB__TOD',
       'EUR000TODTOM', 'USD000TODTOM', 'CNY000000TOD', 'EURUSDTODTOM',
       'EURUSD000TOD', 'EURUSD000TOM', 'USDRUB_TOM1W', 'BKTRUB000TOM',
       'USDRUB_TOM1D', 'USDRUB_TOM1M'], dtype=object)

#### Let's count number of orders per instrument

In [11]:
%time trades.SECCODE.value_counts()

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 10.2 ms


USD000UTSTOM    15530
USD000000TOD     4081
EUR_RUB__TOD     1492
EUR_RUB__TOM     1037
USD000TODTOM      585
EUR000TODTOM      383
EURUSD000TOM       68
CNY000000TOD       44
EURUSDTODTOM       30
EURUSD000TOD       11
BKTRUB000TOM        2
USDRUB_TOM1W        1
USDRUB_TOM1D        1
USDRUB_TOM1M        1
Name: SECCODE, dtype: int64

#### Let's query for a particular instrument put orders

In [13]:
#%time orders[(orders.SECCODE=='USD000UTSTOM') & (orders.BUYSELL=='S')].head(5)
%time trades[(trades.SECCODE=='USD000UTSTOM') & (trades.VOLUME > 100000)].head(5)

CPU times: user 20 ms, sys: 10 ms, total: 30 ms
Wall time: 24 ms


,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME
34,76139220,USD000UTSTOM,1900-01-01 10:01:00.400,34775,34714,29.9895,150000
36,76139222,USD000UTSTOM,1900-01-01 10:01:00.400,34775,34705,29.9900,488000
39,76139225,USD000UTSTOM,1900-01-01 10:01:00.500,21540,36096,29.9830,350000
47,76139235,USD000UTSTOM,1900-01-01 10:01:01.600,40706,40703,29.9795,135000
52,76139240,USD000UTSTOM,1900-01-01 10:01:01.700,40910,40703,29.9795,365000


#### Let's plot a little

In [85]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Spectral10
from bokeh.transform import factor_cmap
from bokeh.models import Range1d, LinearAxis
output_notebook()

Loading BokehJS ...

In [93]:
trade_ticker = trades.SECCODE.value_counts().index.tolist()
trade_values = trades.SECCODE.value_counts().values
print("Index: \n %s \n Values: \n %s" % (trade_ticker, trade_values))

Index: 
 ['USD000UTSTOM', 'USD000000TOD', 'EUR_RUB__TOD', 'EUR_RUB__TOM', 'USD000TODTOM', 'EUR000TODTOM', 'EURUSD000TOM', 'CNY000000TOD', 'EURUSDTODTOM', 'EURUSD000TOD', 'BKTRUB000TOM', 'USDRUB_TOM1W', 'USDRUB_TOM1D', 'USDRUB_TOM1M'] 
 Values: 
 [15530  4081  1492  1037   585   383    68    44    30    11     2     1
     1     1]


In [27]:
gr = figure(plot_width=500, plot_height=500)
gr.vbar(x=ticker_counts.index, top=ticker_counts.values, width=0.5, color="#CAB2D6")
show(gr)

#### Let's build a graph that will show volume (green) and price (red) traded during the day for a particular instrument

In [95]:
plot = figure(title='Number of trades per instrument', x_range=trade_ticker, width=1200)
plot.vbar(trade_ticker,top=trade_values,width=0.5, color='maroon')
plot.xaxis.major_label_orientation = 3.14/2
show(plot)

In [96]:
usd_trades = trades[trades.SECCODE=='USD000UTSTOM'].sort_values(by='TIME')
usd_liner = figure(width=1200, title='Volume (left scale) and Price (right scale) for USD000UTSTOM during the day')
usd_liner.extra_y_ranges = {"price": Range1d(start=usd_trades.PRICE.min(), end=usd_trades.PRICE.max())}
usd_liner.add_layout(LinearAxis(y_range_name="price"), 'right')
usd_liner.line(x=usd_trades.TIME, y=usd_trades.VOLUME, color='green')
usd_liner.line(x=usd_trades.TIME, y=usd_trades.PRICE, color='red', y_range_name='price')
show(usd_liner)